In [1]:
################################################################################
# File name:    "dist_prop_to_station.ipynb"
#
# Project title:    Boston Affordable Housing project (visting scholar porject)
#
# Description:    This file calculates the distance from a property to its 
#                 closest train stop in manhattan and euclidean distance. 
#                 The output is used in the rd_amenities.do file.
#
# Inputs:    ./all_stations.csv
#            ./warren_MAPC_all_unqiue.dta
#
# Outputs:    ./transit_distance.csv
#             ./dist_prop_to_station_log.txt
#
# Created:    06/01/2022
# Updated:    09/29/2022
#
# Author:    Nicholas Chiumenti
################################################################################

In [2]:
import datetime
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.spatial import cKDTree
from shapely.geometry import Point
from math import radians, sin, asin, sqrt, atan2

# Set paths for all inputs/outputs

In [13]:
## set paths throughout program
## CHANGE THESE TO ADJUST WHERE FILES ARE LOADED AND SAVED

# stations file path
stations_path = "/home/a1nfc04/Documents/boston_zoning_sdrive/data/shapefiles/train_stops/all_stations.csv"

# warren group properties data file path
warren_path = "/home/a1nfc04/Documents/boston_zoning_sdrive/data/warren/warren_MAPC_all_unique.dta"

# save path
save_path = "/home/a1nfc04/Documents/boston_zoning_sdrive/data/shapefiles/train_stops/transit_distance.csv"

# log path
log_path = "/home/a1nfc04/Documents/boston_zoning_sdrive/python_programs/transit_distances/dist_prop_to_station_log.txt"

# Load in stations data

In [4]:
# load in stations data
stations_df = pd.read_csv(stations_path)

# drop geometry variable
stations_df.drop(columns = ["geometry"], inplace = True)

# convert to geo data frame
stations_gdf = gpd.GeoDataFrame(stations_df,
                                geometry = gpd.points_from_xy(stations_df['station_lon'], stations_df['station_lat'], crs = "EPSG:4269")
                               )

stations_gdf.rename_geometry('station_geometry', inplace = True)

# re-project
stations_gdf.to_crs("EPSG:26986", inplace=True)

# error check
assert len(stations_gdf) == 303

# Load in the warren group

In [5]:
# load in warren group data, trim variables
warren_df = pd.read_stata(warren_path)
warren_df = warren_df[["prop_id", "cousub_name", "warren_latitude", "warren_longitude"]]

# convert to a geo dataframe
warren_gdf = gpd.GeoDataFrame(warren_df, 
                            geometry = gpd.points_from_xy(warren_df['warren_longitude'], warren_df['warren_latitude'], crs = "EPSG:4269")
                   
                           )

warren_gdf.rename_geometry('warren_geometry', inplace = True)

warren_gdf.to_crs("EPSG:26986", inplace=True)

# error checking
assert len(warren_gdf) == 821237 # check warren group data

# Calculate the nearest neighbor
## get nearest neighbors using Ju-Eun's code

In [6]:
gdfA = warren_gdf
gdfB = stations_gdf

nA = np.array(list(gdfA.warren_geometry.apply(lambda x: (x.x, x.y))))

nB = np.array(list(gdfB.station_geometry.apply(lambda x: (x.x, x.y))))

btree = cKDTree(nB)

# find the nearest neighbor between address points and train stops
dist, idx = btree.query(nA, k=1)

# nearest neighbor for each proeprty
gdfB_nearest = gdfB.iloc[idx].reset_index(drop=True)

assert len(gdfB_nearest) == 821237 # check results

# merge back with the warren group dataset
gdf = pd.concat([gdfA.reset_index(drop=True), gdfB_nearest, pd.Series(dist, name='dist')], axis=1)

final_data = gdf.copy()

final_data['Index'] = final_data.index

# Calculate manhattan distance

In [7]:
## calculating manhattan distance (Ju-Eun's code)
dist_array = []

for i , r in final_data.iterrows():
    # degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, r[["warren_latitude", "warren_longitude", "station_lat", "station_lon"]])

    # Latitude
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    lat_d = c * 6371

    # Longitude
    dlon = lon2 - lon1
    a = sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    lon_d = c * 6371

    dist_array.append((i, (lat_d + lon_d)*1000))

    # print(dist_array)

In [8]:
# export manhattan distance calculations
Manhattan_haversine_distance = pd.DataFrame(dist_array,columns=["origin", "distance(m)"])

Manhattan_haversine_distance["Index"] = Manhattan_haversine_distance["origin"] 

Manhattan_haversine_distance = pd.merge(Manhattan_haversine_distance, final_data, on="Index")

Manhattan_haversine_distance = Manhattan_haversine_distance[["prop_id", "cousub_name", "warren_latitude", "warren_longitude",
                                                             "station_id", "station_name", "station_lat", "station_lon","distance(m)"]]

Manhattan_haversine_distance.rename(columns = {"distance(m)":"distance_m_man"}, inplace = True)
# Manhattan_haversine_distance.to_stata("/home/a1nfc04/boston_zoning_sdrive/data/shapefiles/train_stops/transit_distances_manhattan.dta")

# Calculate euclidian distance

In [9]:
## calculate straight line euclidean distance (Ju-Eun's code)
from math import radians, cos, sin, asin, sqrt

# Euclidean distance between vectors
dist_array_e = []
for i , r in final_data.iterrows():
    lon1, lat1, lon2, lat2 = map(radians, r[["warren_latitude", "warren_longitude", "station_lon", "station_lat"]])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 

    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 

    # Radius of earth in kilometers is 6371
    km = 6371 * c

    dist_array_e.append((i, (km)*1000))

# print(dist_array_e)

In [10]:
# export euclidean distance measures
Euclidean_haversine_distance = pd.DataFrame(dist_array_e,columns=["origin", "distance(m)"])

Euclidean_haversine_distance["Index"] = Euclidean_haversine_distance["origin"] 

Euclidean_haversine_distance = pd.merge(Euclidean_haversine_distance, final_data, on="Index")

Euclidean_haversine_distance = Euclidean_haversine_distance[["prop_id", "cousub_name", "warren_latitude", "warren_longitude",
                                                             "station_id", "station_name", "station_lat", "station_lon","distance(m)"]]

Euclidean_haversine_distance.rename(columns = {"distance(m)":"distance_m_euc"}, inplace = True)

# Euclidean_haversine_distance.to_csv("/home/a1nfc04/boston_zoning_sdrive/data/shapefiles/train_stops/transit_distances_euclid.csv")

In [14]:
merged_df = Euclidean_haversine_distance.merge(Manhattan_haversine_distance, left_index = True, right_index = True)

merged_df = merged_df[["prop_id_x", "cousub_name_x", "warren_latitude_x", "warren_longitude_x",
                      "station_id_x", "station_name_x", "station_lat_x", "station_lon_x",
                      "distance_m_euc", "distance_m_man"]]

merged_df.columns = ["prop_id", "cousub_name", "warren_latitude", "warren_longitude",
                      "station_id", "station_name", "station_lat", "station_lon",
                      "distance_m_euc", "distance_m_man"]

merged_df.to_csv(save_path, index = False)

In [15]:
# save a log .txt file to the S drive
date = datetime.datetime.now().strftime('%D at %I:%M:%S %p')

with open(log_path,'a') as file:
    file.write(f"Finish running on {date}: {len(merged_df):,} observations written to '{save_path}'.\n")  

# Done!
print(f"Done! {len(merged_df):,} observations written")

Done! 821,237 observations written
